In [1]:
import os
os.getcwd()

'c:\\Users\\nmishra\\OneDrive - The Scripps Research Institute\\projects\\taq_ligase'

In [6]:
step_number = "3b"  # Example model number

In [ ]:
# Construct paths based on the model number
# to be updated with construct_path function from plotting notebook
cif_directory = f"../taq/af3preds/step_{step_number}/extracted/"
main_folder = f"../taq/af3preds/step_{step_number}/pdbs/"
save_path = f"../taq/af3preds/step_{step_number}/rmsf.png"
csv_path = f"../taq/af3preds/step_{step_number}/rmsf.csv"

print(f"Working on Step {step_number}:")
print(f"CIF directory: {cif_directory}")
print(f"PDB folder: {main_folder}")
print(f"Save path for image: {save_path}")

Working on Step 3b:
CIF directory: taq/af3preds/step_3b/extracted/
PDB folder: taq/af3preds/step_3b/pdbs/
Save path for image: taq/af3preds/step_3b/rmsf.png


In [57]:
from Bio.PDB.MMCIFParser import MMCIFParser
from Bio.PDB.PDBIO import PDBIO
import os
import glob


def convert_cif_to_pdb(main_dir):
    # Define the output directory for PDB files
    output_dir = os.path.join(main_dir, "..", "pdbs")
    os.makedirs(output_dir, exist_ok=True)

    # Iterate over each subdirectory
    for subdir in os.listdir(main_dir):
        subdir_path = os.path.join(main_dir, subdir)

        # Check if it's a directory
        if os.path.isdir(subdir_path):
            print(f"Processing directory: {subdir_path}")

            # List all CIF files in the subdirectory
            cif_files = glob.glob(os.path.join(subdir_path, "*.cif"))

            # Convert each CIF file to PDB format
            for cif_file in cif_files:
                try:
                    # Parse CIF file
                    parser = MMCIFParser()
                    structure = parser.get_structure(
                        os.path.basename(cif_file), cif_file
                    )

                    # Generate PDB file name in the output directory
                    pdb_filename = (
                        os.path.splitext(os.path.basename(cif_file))[0] + ".pdb"
                    )
                    pdb_file = os.path.join(output_dir, pdb_filename)

                    # Save structure as PDB file
                    pdb_io = PDBIO()
                    pdb_io.set_structure(structure)
                    pdb_io.save(pdb_file)

                    print(f"Converted {cif_file} to {pdb_file}")
                except Exception as e:
                    print(f"Error converting {cif_file}: {str(e)}")

# add a check to see if the PDB folder is empty
# if empty, then run the converter, else skip
if not os.listdir(main_folder):
    convert_cif_to_pdb(cif_directory)

In [58]:
# Calculate RMSF of ensemble of predictions for each subsampling condition

# Select atoms for RMSF calculations
rmsf_sel = "backbone"

import os
import glob
import re
from typing import List, Tuple, Union

import pandas as pd
import numpy as np
import warnings

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.cm as cm
import seaborn as sns

import MDAnalysis as mda
from MDAnalysis.analysis import pca, align, rms

from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

warnings.filterwarnings("ignore")


class LoadPredictions:
    def __init__(self, trajectory_files: List[str], topology_file: str) -> None:
        """
        Initialize the LoadTrajectory class with a list of trajectory files and a single topology file.

        :param trajectory_files: List of paths to trajectory files (.pdb)
        :param topology_file: Path to the topology file (.pdb)
        """
        self.trajectory_files = trajectory_files
        self.topology_file = topology_file
        self.universe: Union[None, mda.Universe] = None

    def load(self) -> None:
        """
        Load the trajectories and the topology file into an MDAnalysis Universe.
        """
        try:
            self.universe = mda.Universe(
                self.topology_file, self.trajectory_files, dt=1
            )
        except Exception as e:
            print(f"An error occurred while loading the files: {e}")

    def get_universe(self) -> Union[mda.Universe, None]:
        """
        Return the MDAnalysis Universe object.
        """
        if self.universe is not None:
            return self.universe
        else:
            print("Universe not loaded. Call the 'load' method first.")
            return None


def atoi(text: str) -> Union[int, str]:
    return int(text) if text.isdigit() else text


def natural_keys(text: str) -> List[Union[int, str]]:
    """
    Helper function for natural sorting (human sorting).
    """
    return [atoi(c) for c in re.split(r"(\d+)", text)]


def sorted_natural_glob(pattern: str, sort: bool = False) -> List[str]:
    """
    Perform a glob search and return a list of file paths.
    Sort the list naturally by file name if sort is True.

    :param pattern: Glob pattern to search for.
    :param sort: Boolean indicating whether to sort the results.
    :return: List of file paths, sorted if sort is True.
    """
    file_paths: List[str] = glob.glob(pattern)
    if sort:
        file_paths.sort(key=lambda x: natural_keys(x))
    return file_paths


def ensembleRMSF(
    base_path: str, analysis_range: str, sort: bool = True
) -> Tuple[List[rms.RMSF], mda.AtomGroup]:
    pdb_files = sorted_natural_glob(os.path.join(base_path, "*.pdb"), sort)
    if not pdb_files:
        print(f"No PDB files found in {base_path}")
        return None, None

    topology_file = pdb_files[0]

    u_getter = LoadPredictions(pdb_files, topology_file)
    u_getter.load()
    u = u_getter.get_universe()
    if u is None:
        return None, None

    average = align.AverageStructure(u, u, select=analysis_range, ref_frame=0).run()
    ref = average.results.universe
    aligner = align.AlignTraj(u, ref, select=analysis_range, in_memory=True).run()
    c_alphas = u.select_atoms(analysis_range)
    R = rms.RMSF(c_alphas).run()
    return R, c_alphas

# Initialize the results list
rmsf_results: List[Tuple[str, rms.RMSF]] = []

In [59]:
R, c_alphas = ensembleRMSF(main_folder, rmsf_sel)

In [60]:
first_plotted_residue = min(c_alphas.resids)  # @param {type:"string"}
print(first_plotted_residue)
last_plotted_residue = max(c_alphas.resids)  # @param {type:"string"}
print(last_plotted_residue)

1
676


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def plot_rmsf(c_alphas, rmsf_results, save_path):
    fig_size = (7, 2)
    plt.figure(figsize=fig_size)

    plt.plot(c_alphas.resids, rmsf_results.rmsf, color="gray", linewidth=0.5)
    plt.xlabel("Residue number")
    plt.ylabel("RMSF ($\AA$)")

    # Define regions and their properties
    regions = [
        {"start": 34, "end": 38, "color": "#FF9671", "label": "AdD"},
        {"start": 84, "end": 85, "color": "#FF9671", "label": "AdD"},
        {"start": 116, "end": 117, "color": "#FF9671", "label": "AdD"},
        {"start": 139, "end": 140, "color": "#FF9671", "label": "AdD"},
        {"start": 174, "end": 175, "color": "#FF9671", "label": "AdD"},
        {"start": 294, "end": 295, "color": "#FF9671", "label": "AdD"},
        {"start": 318, "end": 320, "color": "#FF9671", "label": "AdD"},
        {"start": 408, "end": 432, "color": "#2C73D2", "label": "ZNB"},
        {"start": 334, "end": 338, "color": "#008F7A", "label": "OBD"},
        {"start": 454, "end": 459, "color": "#008F7A", "label": "OBD"},
        {"start": 520, "end": 525, "color": "#008F7A", "label": "OBD"},
        {"start": 552, "end": 557, "color": "#008F7A", "label": "OBD"},
        {"start": 472, "end": 548, "color": "#845EC2", "label": "HhH"},
        {"start": 589, "end": 676, "color": "#D65DB1", "label": "BRCT"},
    ]

    # Plot regions
    for region in regions:
        plt.axvspan(
            region["start"],
            region["end"],
            zorder=0,
            alpha=0.2,
            color=region["color"],
            label=region["label"],
        )

    # Customize legend
    handles, labels = plt.gca().get_legend_handles_labels()
    unique_labels = list(set(labels))  # Get unique labels
    plt.legend(
        [handles[labels.index(label)] for label in unique_labels],
        unique_labels,
        bbox_to_anchor=(0.5, -0.35),
        loc="upper center",
        ncol=5,
        frameon=False,
    )

    plt.ylim(0, 30)
    plt.yticks(np.arange(0, 31, 5))
    ax = plt.gca()
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["bottom"].set_visible(True)
    ax.spines["bottom"].set_linewidth(0.25)

    ax.tick_params(
        width=0.25,
        length=10,
        color="#4B4453",
        pad=10,
        labelsize=12,
        labelcolor="#4B4453",
    )

    plt.savefig(save_path, dpi=600, bbox_inches="tight")


# Example usage:
# Assuming c_alphas and R.results are defined elsewhere
# c_alphas = ...
# R.results = ...
# save_path = "path/to/save/figure.png"

# Call the function to generate the plot
plot_rmsf(c_alphas, R.results, save_path)

In [62]:
# Save data to CSV
data = {"Residue_number": c_alphas.resids, "RMSF": R.results.rmsf}
df = pd.DataFrame(data)
df.to_csv(csv_path, index=False)